In [4]:
import math
import random
import numpy as np
import os
import pandas as pd

In [5]:
def get_hills_by_sum(cat_nof, sample2cat, sample2snp, snp_list, gene2snp):
        snp_count = dict()
        for snp in snp_list:
            snp_count[snp] = { c:0 for c in cat_nof.keys() }
        for sample,snps in sample2snp.items():
            for snp in snps:
                snp_count[snp][sample2cat[sample]] += 1
                
        hills = {gene:dict() for gene in gene2snp.keys()}
        for gene in gene2snp.keys():
            for cat in cat_nof.keys():
                hills[gene][cat] = 0
            for snp in gene2snp[gene]:
                for cat in cat_nof.keys():
                    hills[gene][cat] += snp_count[snp][cat] / (cat_nof[cat] * len(gene2snp[gene]))
        return hills

In [8]:
for adni in ['ADNI1','ADNI2_GO','ADNI3']:
    folder = '/Users/guglielmo/Desktop/Entropy/DataSet/'+adni+'/'
    print()
    print(adni)
    !mkdir {folder + 'EmpiricPvalue_3_classes_1000'}
    
    ad  = !wc -l < {folder+'/Subjects/'+'AD.fam'}
    ad  = int(ad.fields(0)[0])

    cn  = !wc -l < {folder+'/Subjects/'+'CN.fam'}
    cn  = int(cn.fields(0)[0])

    mci  = !wc -l < {folder+'/Subjects/'+'MCI.fam'}
    mci  = int(mci.fields(0)[0])
    
    path_reduced = folder+'/Subjects/'+'Plink_reduced'
    
    snp2gene = dict()
    gene2snp = dict()
    iline = 0
    for line in open(os.path.join(folder, 'SnpToGene_RSID.txt')):
        if iline > 0:
            cc = line.strip().split('\t')
            gene = cc[0]
            if gene not in gene2snp:
                gene2snp[gene] = list()
            snps = cc[3].split(',')
            for snp in snps:
                snp = snp.strip()
                snp2gene[snp] = gene
                gene2snp[gene].append(snp)
        iline += 1

    cats = ['AD','CN','MCI']
    cat_nof = {'AD':ad,'CN':cn,'MCI':mci}
    sample2cat = (['AD']*ad) + (['CN']*cn) + (['MCI']*mci)

    sample2snp = dict()
    snp_list = list()

    iline = 0
    for line in open(os.path.join(path_reduced, 'merged.raw')):
        if iline == 0:
            snp_list = [v[:-2] for v in  line.strip().split(' ')]
        else:
            cc = line.strip().split(' ')
            sample2snp[iline-1] = [snp_list[i] for i in range(len(cc)) if cc[i]=='1']

        iline += 1

    nof_rands = 1000
    random.seed(0)

    shuffle_perc = 0.5
    file_name = 'Intragenic_log_' + str(round(shuffle_perc,2)) + '.txt'
    f = open(os.path.join(folder, "EmpiricPvalue_3_classes_1000", file_name), 'a+')  
    
    w = ["Cat1","Cat2","Gene","FC","Pvalue", "HillCat1" , "HillCat2"]
    w = ' '.join(w)
    f.write(w + '\n')
    
    nof_shuffs = math.ceil( len(sample2cat)*shuffle_perc )
    real_hills = get_hills_by_sum(cat_nof, sample2cat, sample2snp, snp_list, gene2snp)
    
    rand_hills = {gene:dict() for gene in gene2snp.keys()}
    for gene in gene2snp.keys():
        for cat in cat_nof.keys():
            rand_hills[gene][cat] = [0 for i in range(nof_rands)]

    for r in range(nof_rands):

        if((r+1) % 100) == 0:
            print(r+1)

        rand_sample2cat = [x for x in sample2cat]
        for s in range( nof_shuffs ):
            s1 = random.randint(0,len(rand_sample2cat)-1)
            s2 = random.randint(0,len(rand_sample2cat)-1)
            c = rand_sample2cat[s1]
            rand_sample2cat[s1] = rand_sample2cat[s2]
            rand_sample2cat[s2] = c

        # To delete ?
        adc = 0
        mcic = 0
        for i in range(len(sample2cat)):
            if (sample2cat[i] == 'AD') and (rand_sample2cat[i] == 'AD'):
                adc += 1
            if (sample2cat[i] == 'MCI') and (rand_sample2cat[i] == 'MCI'):
                mcic += 1

        hills = get_hills_by_sum(cat_nof, rand_sample2cat, sample2snp, snp_list, gene2snp)
        for gene in hills.keys():
            for cat in cat_nof.keys():
                rand_hills[gene][cat][r] = hills[gene][cat]


    counts = dict()
    count_nozero = dict()

    for cat1_i in range(len(cats)):
        for cat2_i in range(cat1_i+1, len(cats)):
            cat1 = cats[cat1_i]
            cat2 = cats[cat2_i]
            cca = cat1+'_'+cat2
            counts[cca] = 0
            count_nozero[cca] = 0

            for gene in sorted(real_hills.keys()):
                a = rand_hills[gene][cat1]
                b = rand_hills[gene][cat2]
                diff_rand = [ abs(math.log(a[i]+1) - math.log(b[i]+1)) for  i in range(len(a)) ]
                diff_real = abs(math.log(real_hills[gene][cat1]+1) - math.log(real_hills[gene][cat2]+1))
                pvalue = 0.0
                for i in range(len(diff_rand)):
                    if diff_rand[i] >= diff_real:
                        pvalue += 1.0
                pvalue = pvalue / float(len(diff_rand))

                if pvalue <= 0.05:
                    counts[cca] += 1
                    w = [cat1,cat2,gene,str(diff_real),str(pvalue),str(real_hills[gene][cat1]),str(real_hills[gene][cat2])]
                    w = ' '.join(w)
                    f.write(w + '\n')
                if diff_real > 0.0:
                    count_nozero[cca] += 1
    f.close()


ADNI1
mkdir: /Users/guglielmo/Desktop/Entropy/DataSet/ADNI1/EmpiricPvalue_3_classes_1000: File exists
100
200
300
400
500
600
700
800
900
1000

ADNI2_GO
mkdir: /Users/guglielmo/Desktop/Entropy/DataSet/ADNI2_GO/EmpiricPvalue_3_classes_1000: File exists
100
200
300
400
500
600
700
800
900
1000

ADNI3
mkdir: /Users/guglielmo/Desktop/Entropy/DataSet/ADNI3/EmpiricPvalue_3_classes_1000: File exists
100
200
300
400
500
600
700
800
900
1000
